## Libary Set-up

In [1]:
import numpy as np
import pandas as pd
from PIL import Image
import os
import warnings
import rasterio as rio

%matplotlib inline

from netCDF4 import Dataset
from pyhdf.SD import SD, SDC
from skimage.transform import resize

In [2]:
from collections import Counter

from sklearn.utils import shuffle
from sklearn.pipeline import Pipeline
from sklearn.metrics import  confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from skimage.transform import resize
from sklearn.metrics import mean_squared_error

ncores = os.cpu_count()

## 2010 cropland boundaries - generate Crop mask

In [3]:
lidar_dem_path = 'lower_scaled_gfsad.tif'
with rio.open(lidar_dem_path) as lidar_dem:
    im_array = lidar_dem.read()

In [4]:
print(im_array.shape)
# Get a list of cropland and their classes
im_array = im_array.reshape((2160,4320))

def apply_mask(pixel):
    if pixel == 9:
        return 0
    else:
        return pixel

filter_function = np.vectorize(apply_mask)
unmasked_pixels = filter_function(im_array)

land_pixels = np.nonzero(unmasked_pixels) 
# print(np.unique(imarray))
land_pixel_classes = im_array[land_pixels].tolist()
print(len(land_pixel_classes))

(1, 2160, 4320)
702138


# Produce Data_frame

In [5]:
land_indices = land_pixels 
non_zero_indices = np.array(land_indices)
clean_frame = non_zero_indices.T
print(clean_frame.shape)
n = clean_frame.shape[0]
non_zeros = np.nonzero(im_array)

clean_frame_2 = clean_frame 
clean_frame_df = pd.DataFrame({'lon': clean_frame[:,0], 'lat': clean_frame[:,1]})
clean_frame_df['labels'] = land_pixel_classes

(702138, 2)


## Add siebert labels
Each year will take the cropland mask and produce values for the pixel for each year. 

In [29]:
label_years = ['1985', '1990', '1995', '2000', '2005']
output_years = label_years + [ '2001', '2002', '2003', '2004', '2006','2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015',
  '2016', '2017', '2018', '2019']
lidar_dem_path = 'C:\\Users\\Elle\\Documents\\w210\\1985.tif'
times_series_labels = np.zeros((5, 2160, 4320))
for i in range(len(label_years)):
    lidar_dem_path = label_years[i] +'.tif'
    with rio.open(lidar_dem_path) as lidar_dem:
        array = lidar_dem.read() 
        array = array.reshape(2160, 4320)
        clean_frame_df[str(label_years[i])] = array[land_indices].reshape(n,1)

times_series_labels[0].shape

(2160, 4320)

## Retrieve NDVI Data for each year

In [30]:
measures = ['max_y1', 'min_y1', 'mean_y1', 'var_y1', 'max_y2', 'min_y2', 'mean_y2', 'var_y2']
ndvi_list = os.listdir('ndvi')
def retrieve_ndvi(indices, length, year):
            path = 'ndvi/ndvi3g_geo_v1_'
            file_1h = path + year + '_0106.nc4'
            file_2h = path + year + '_0712.nc4' 
            ds_1, ds_2 = np.array(Dataset(file_1h)['ndvi']) , np.array(Dataset(file_2h)['ndvi'])
            max_y1 = np.max(ds_1, axis = 0)[indices].reshape(length)
            min_y1 = np.min(ds_1, axis = 0)[indices].reshape(length)
            var_y1 = np.var(ds_1, axis = 0)[indices].reshape(length)
            mean_y1= np.mean(ds_1, axis = 0)[indices].reshape(length)
            max_y2 = np.max(ds_2, axis = 0)[indices].reshape(length)
            min_y2 = np.min(ds_2, axis = 0)[indices].reshape(length)
            var_y2 = np.var(ds_2, axis = 0)[indices].reshape(length)
            mean_y2 = np.mean(ds_2, axis = 0)[indices].reshape(length)
            return max_y1, min_y1, mean_y1, var_y1, max_y2, min_y2, mean_y2, var_y2

In [20]:
data.max()

0

In [33]:
def ndvi_mask(pixel):
    if pixel < -1:
        return 0
    else:
        return pixel*10000

def ndvi_modis(indices, length, year):
    ndvi_frame = np.zeros((23,2160,4320))
    file_dir = 'daily_ndvi/processed/' + year + '/'
    for i in range(23):
        full_path = file_dir + str(i) + '.csv' 
        ds = pd.read_csv(full_path)
        filter_function = np.vectorize(ndvi_mask)
        data = filter_function(ds)
        data = resize(data, (2160, 4320), preserve_range=True)
        ndvi_frame[i,:,:]= data
    ds_1, ds_2  = ndvi_frame[0:12,:,:], ndvi_frame[12:,:,:]
    max_y1 = np.max(ds_1, axis = 0)[indices].reshape(length)
    min_y1 = np.min(ds_1, axis = 0)[indices].reshape(length)
    var_y1 = np.var(ds_1, axis = 0)[indices].reshape(length)
    mean_y1= np.mean(ds_1, axis = 0)[indices].reshape(length)
    max_y2 = np.max(ds_2, axis = 0)[indices].reshape(length)
    min_y2 = np.min(ds_2, axis = 0)[indices].reshape(length)
    var_y2 = np.var(ds_2, axis = 0)[indices].reshape(length)
    mean_y2 = np.mean(ds_2, axis = 0)[indices].reshape(length)
    return max_y1, min_y1, mean_y1, var_y1, max_y2, min_y2, mean_y2, var_y2

## Retrieve Climate Data for each year

In [34]:
def extract_nc(indices, length, year, variable):
    path = 'climate/' + year + '/' + variable + '/'
    full_path = path + 'TerraClimate_' + variable +'_' + year + '.nc'
    ds = np.array(Dataset(full_path)[variable])
    max_y1 = np.max(ds, axis = 0)
    min_y1 = np.min(ds, axis = 0)
    var_y1 = np.var(ds, axis = 0)
    mean_y1 = np.mean(ds, axis = 0)
    max_y1 = resize(max_y1, (2160, 4320))[indices].reshape(length)
    min_y1 = resize(min_y1, (2160, 4320))[indices].reshape(length)
    var_y1 = resize(var_y1, (2160, 4320))[indices].reshape(length)
    mean_y1 = resize(mean_y1, (2160, 4320))[indices].reshape(length)
    return max_y1, min_y1, var_y1, mean_y1


## Save each Year to a CSV

In [36]:
def retrieve_year_clim(indices, length, year, ref_df):
    measures = ['aet', 'def', 'pet', 'ppt', 'srad', 'tmax', 'tmin', 'vap', 'vpd', 'soil', 'PDSI']
    train_years = ['1985', '1990', '1995', '2000', '2005']
    
    
    new_df = pd.DataFrame()
    new_df['lat'], new_df['lon']  = ref_df['lat'], ref_df['lon']
    
    #Add labels - where applicable 
    if year in train_years:
        new_df['label'] = ref_df[year]
    
    #retrieve ndvi
#     if int(year) < 2016:
#         #if avhrr is available
#         new_df['ndvi_max_y1'], new_df['ndvi_min_y1'], new_df['ndvi_mean_y1'], new_df['ndvi_var_y1'], new_df['ndvi_max_y2'], new_df['ndvi_min_y2'], new_df['ndvi_mean_y2'], new_df['ndvi_var_y2']  = retrieve_ndvi(indices, length, year)
#     else:
        #else use modis
    new_df['ndvi_max_y1'], new_df['ndvi_min_y1'], new_df['ndvi_mean_y1'], new_df['ndvi_var_y1'], new_df['ndvi_max_y2'], new_df['ndvi_min_y2'], new_df['ndvi_mean_y2'], new_df['ndvi_var_y2']  = ndvi_modis(indices, length, year)
    print('ndvi done for' + year)
    for i in measures:
        #retrieve relevant climate variables
        new_df[i+'_max'], new_df[i+'_min'], new_df[i+'_var'], new_df[i+'_mean'] = extract_nc(indices, length, year, i)
    print('climate' + year)
    return new_df

for j in ['2019']:
    t= retrieve_year_clim(land_indices, n, j , clean_frame_df)
    file_save_loc = 'data/new_ndvi/' + j + '.csv'
    t.to_csv(file_save_loc)

ndvi done for2019
climate2019


## Extract Long term averages

In [6]:
measures_2 = ['aet', 'def', 'pet', 'ppt', 'srad', 'tmax', 'tmin', 'vap', 'vpd', 'soil']  

def extract_nc_lt(indices, length, variable):
    path = 'climate/lt/' + variable + '/'
    full_path = path + 'TerraClimate19812010_' + variable  + '.nc'
    print(full_path)
    ds = np.array(Dataset(full_path)[variable])
    max_y1 = np.max(ds, axis = 0)
    min_y1 = np.min(ds, axis = 0)
    var_y1 = np.var(ds, axis = 0)
    mean_y1= np.mean(ds, axis = 0)
    max_y1 = resize(max_y1, (2160, 4320))[indices].reshape(length,1)
    min_y1 = resize(min_y1, (2160, 4320))[indices].reshape(length,1)
    var_y1 = resize(var_y1, (2160, 4320))[indices].reshape(length,1)
    mean_y1 = resize(mean_y1, (2160, 4320))[indices].reshape(length,1)
    return max_y1, min_y1, var_y1, mean_y1

for i in measures_2:
    clean_frame_df[i+'_lt_max'], clean_frame_df[i+'_lt_min'], clean_frame_df[i+'_lt_var'], clean_frame_df[i+'_lt_mean'] = extract_nc_lt(land_indices, n, i)

climate/lt/aet/TerraClimate19812010_aet.nc
climate/lt/def/TerraClimate19812010_def.nc
climate/lt/pet/TerraClimate19812010_pet.nc
climate/lt/ppt/TerraClimate19812010_ppt.nc
climate/lt/srad/TerraClimate19812010_srad.nc
climate/lt/tmax/TerraClimate19812010_tmax.nc
climate/lt/tmin/TerraClimate19812010_tmin.nc
climate/lt/vap/TerraClimate19812010_vap.nc
climate/lt/vpd/TerraClimate19812010_vpd.nc
climate/lt/soil/TerraClimate19812010_soil.nc


In [7]:
lt_frame = clean_frame_df[['lon', 'lat','aet_lt_max', 'aet_lt_min', 'aet_lt_var', 'aet_lt_mean', 'def_lt_max',
       'def_lt_min', 'def_lt_var', 'def_lt_mean', 'pet_lt_max', 'pet_lt_min',
       'pet_lt_var', 'pet_lt_mean', 'ppt_lt_max', 'ppt_lt_min', 'ppt_lt_var',
       'ppt_lt_mean', 'srad_lt_max', 'srad_lt_min', 'srad_lt_var', 'soil_lt_max', 'soil_lt_min', 'soil_lt_var',
       'soil_lt_mean',
       'srad_lt_mean', 'tmax_lt_max', 'tmax_lt_min', 'tmax_lt_var',
       'tmax_lt_mean', 'tmin_lt_max', 'tmin_lt_min', 'tmin_lt_var',
       'tmin_lt_mean', 'vap_lt_max', 'vap_lt_min', 'vap_lt_var', 'vap_lt_mean',
       'vpd_lt_max', 'vpd_lt_min', 'vpd_lt_var', 'vpd_lt_mean' ]]
lt_frame.to_csv('data/lt.csv')

## Compare NDVI for the same year

In [9]:
path = 'ndvi/ndvi3g_geo_v1_'
file_1h = path + '2015' + '_0106.nc4'
ds_3 = np.array(Dataset(file_1h)['ndvi'])
max_y3 = np.max(ds_3, axis = 0)[land_indices].reshape(n)
min_y3 = np.min(ds_3, axis = 0)[land_indices].reshape(n)

/home/ubuntu/anaconda3/envs/w210-dlvm/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: WARNING: valid_range not used since it
cannot be safely cast to variable data type
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
year = '2015'
ndvi_frame = np.zeros((23,2160,4320))
file_dir = 'evi/' + year + '/'
file_list = os.listdir(file_dir)
for i in range(len(file_list)):
    ndvi_file = SD(file_dir + file_list[i], SDC.READ)
    sds_obj = ndvi_file.select('CMG 0.05 Deg 16 days NDVI') 
    data = np.array(sds_obj.get()) # get sds data
    data = resize(data, (2160, 4320), preserve_range=True)
    ndvi_frame[i,:,:]= data
    ds_1, ds_2  = ndvi_frame[0:12,:,:], ndvi_frame[12:,:,:]
    max_y1 = np.max(ds_1, axis = 0)[land_indices].reshape(n)
    min_y1 = np.min(ds_1, axis = 0)[land_indices].reshape(n)

HDF4Error: SD (15): File is supported, must be either hdf, cdf, netcdf

In [ ]:
print(min_y1.mean())
print(min_y3.mean())